# Working with the Transient Class

This notebook will go through working with the methods of the transient class and how they will be useful. First we will setup the otter connection, if this doesn't look familiar to you, you may want to return to the [basic_usage.ipynb](./basic_usage.ipynb) notebook!

### Setup

In [1]:
# imports
import os
import otter

from astropy.coordinates import SkyCoord
from astropy import units as u
import pandas as pd

import matplotlib.pyplot as plt

In [2]:
# THIS IS WHAT YOU CHANGE IF YOU HAVE THE DATA IN A DIFFERENT PATH
otterpath = os.path.join(os.environ['OTTER_DATA_DIR']) 

# connect to the dataset
db = otter.Otter(otterpath)
summary = db.generate_summary_table(save=True)

### The `Transient` Class

The first thing I will do is display the documentation for this class so you can skim through the methods. Although, for more details, and a cleaner appearance, see [https://astro-otter.readthedocs.io](https://astro-otter.readthedocs.io).

In [3]:
help(otter.Transient)

Help on class Transient in module otter.io.transient:

class Transient(collections.abc.MutableMapping)
 |  Transient(d={}, name=None)
 |  
 |  Method resolution order:
 |      Transient
 |      collections.abc.MutableMapping
 |      collections.abc.Mapping
 |      collections.abc.Collection
 |      collections.abc.Sized
 |      collections.abc.Iterable
 |      collections.abc.Container
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __add__(self, other, strict_merge=True)
 |      Merge this transient object with another transient object
 |      
 |      Args:
 |          other [Transient]: A Transient object to merge with
 |          strict_merge [bool]: If True it won't let you merge objects that
 |                               intuitively shouldn't be merged (ie. different
 |                               transient events).
 |  
 |  __delitem__(self, keys)
 |  
 |  __getitem__(self, keys)
 |      Override getitem to recursively access Transient elements
 |  
 |  __in

The particularly useful methods here are the getters which return what we have deemed to be the "default" value of a property. 

First though, we must grab a transient from the OTTER dataset. Let's use ASASSN-14li since it has a pretty solid dataset.

In [4]:
t = db.query(names='ASASSN-14li')[0] # if you don't know why I use [0] go back to the basic_usage tutorial

t

Transient(
	Name: ASASSN-14li,
	Keys: dict_keys(['distance', 'photometry', 'name', 'reference_alias', 'filter_alias', 'coordinate', 'classification', 'date_reference', 'host', 'schema_version'])
)

And now that next few cells will demonstrate some of the more helpful getter methods and instance variables

#### Name

In [5]:
t.default_name

'ASASSN-14li'

#### Coordinate

Note that this returns an astropy SkyCoord

In [6]:
t.get_skycoord()

<SkyCoord (ICRS): (ra, dec) in deg
    (192.06343875, 17.77402083)>

#### Classification

This returns a tuple of (classification, our confidence, reference list)

In [7]:
classification, conf, refs = t.get_classification()
print(f'{t.default_name} is classified as {classification} with a confidence of {conf}')
print(f'This has been confirmed by the following bibcodes:')
for b in refs:
    print(f'\t-{b}')

ASASSN-14li is classified as TDE with a confidence of 1.0
This has been confirmed by the following bibcodes:
	-2012PASP..124..668Y
	-2014ATel.6777....1J
	-2015Natur.526..542M
	-2016ApJ...819L..25A
	-2016ApJ...832L..10R
	-2016Sci...351...62V
	-2018MNRAS.475.4011B
	-2023PASP..135c4101G
	-2024MNRAS.527.2452M
	-ASAS-SN Supernovae


#### Redshift

In [8]:
t.get_redshift()

'0.0206'

#### Discovery Date

In [9]:
t.get_discovery_date()

<Time object: scale='utc' format='mjd' value=56983.0>

#### Photometry
Since the Transient object only has the unconverted photometry we recommend you use the `clean_photometry` method to convert everything appropriately.

In [10]:
t.clean_photometry()

/home/nfranz/.local/lib/anaconda3/lib/python3.11/site-packages/astropy/units/function/logarithmic.py:67: RuntimeWarning: invalid value encountered in log10
  return dex.to(self._function_unit, np.log10(x))


,reference,raw,raw_units,date,date_format,filter_key,computed,obs_type,upperlimit,corr_k,...,human_readable_refs,converted_wave,converted_wave_unit,converted_freq,converted_freq_unit,converted_flux,converted_flux_err,converted_flux_unit,converted_date,converted_date_unit
624,2016Sci...351...62V,1.93,mJy,57014.08,mjd,15.7GHz,NaN,radio,False,False,...,van Velzen et al. (2016),1.909506e+07,nm,1.570000e+01,GHz,15.686107,0.894027,mag(AB),57014.08,MJD
625,2016Sci...351...62V,1.95,mJy,57015.17,mjd,15.7GHz,NaN,radio,False,False,...,van Velzen et al. (2016),1.909506e+07,nm,1.570000e+01,GHz,15.674913,1.044994,mag(AB),57015.17,MJD
626,2016Sci...351...62V,1.98,mJy,57017.10,mjd,15.7GHz,NaN,radio,False,False,...,van Velzen et al. (2016),1.909506e+07,nm,1.570000e+01,GHz,15.658337,0.869908,mag(AB),57017.10,MJD
627,2016Sci...351...62V,2.1,mJy,57021.11,mjd,15.7GHz,NaN,radio,False,False,...,van Velzen et al. (2016),1.909506e+07,nm,1.570000e+01,GHz,15.594452,0.891112,mag(AB),57021.11,MJD
628,2016Sci...351...62V,1.58,mJy,57039.09,mjd,15.7GHz,NaN,radio,False,False,...,van Velzen et al. (2016),1.909506e+07,nm,1.570000e+01,GHz,15.903357,0.905887,mag(AB),57039.09,MJD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,[2017ApJ...838..149A],0.0791,ct,57364.5,MJD,0.3 - 2.0,NaN,xray,False,None,...,"Auchettl, Guillochon, & Ramirez-Ruiz (2017)",7.293188e-01,nm,4.110582e+08,GHz,25.780686,0.609480,mag(AB),57364.50,MJD
17,[2017ApJ...838..149A],0.0717,ct,57425.5,MJD,0.3 - 2.0,NaN,xray,False,None,...,"Auchettl, Guillochon, & Ramirez-Ruiz (2017)",7.293188e-01,nm,4.110582e+08,GHz,25.887330,0.964005,mag(AB),57425.50,MJD
18,[2017ApJ...838..149A],0.0424,ct,57538.5,MJD,0.3 - 2.0,NaN,xray,False,None,...,"Auchettl, Guillochon, & Ramirez-Ruiz (2017)",7.293188e-01,nm,4.110582e+08,GHz,26.457713,1.179365,mag(AB),57538.50,MJD
19,[2017ApJ...838..149A],0.664,ct,54818.8,MJD,0.3 - 2.0,NaN,xray,True,None,...,"Auchettl, Guillochon, & Ramirez-Ruiz (2017)",7.293188e-01,nm,4.110582e+08,GHz,26.085101,NaN,mag(AB),54818.80,MJD


#### Host Information

This will return a list of otter `Host` objects. See the tutorial [host_objects.ipynb](./host_objects.ipynb) for more details on the functionality we provide here.

The Host objects only store metadata on the host like redshift, ra, dec, and name. If a Host is not in OTTER, we attempt to find the best matching hosts using [astro-ghost](https://uiucsnastro-ghost.readthedocs.io/en/latest/).

In [11]:
hlist = t.get_host()
h0 = hlist[0]

h0, type(hlist), type(h0)

(SDSS J124815.23+174626.4 @ (RA, Dec)=(192.06249999999997 deg,17.77401111111111 deg),
 list,
 otter.io.host.Host)

### Advanced Usage of the Transient Class

Say you want other information that is not easily accessible by the getters shown above. Or, you think we are wrong about the default value for that property. You can then just treat the Transient object like a python dictionary to access the other values for those properties yourself.

First, it has a `keys` method, so let's start there.

In [12]:
t.keys()

dict_keys(['distance', 'photometry', 'name', 'reference_alias', 'filter_alias', 'coordinate', 'classification', 'date_reference', 'host', 'schema_version'])

Let's say you want to see what other distance measurements exist for ASASSN-14li besides the redshift. Let's check that property

In [13]:
t['distance']

[{'value': '0.0205778',
  'distance_type': 'redshift',
  'reference': ['2015Natur.526..542M', '2024MNRAS.527.2452M'],
  'default': False},
 {'value': '0.0206',
  'distance_type': 'redshift',
  'reference': ['2012PASP..124..668Y',
   '2014ATel.6777....1J',
   '2015arXiv150701598H',
   '2016ApJ...819L..25A',
   '2016ApJ...832L..10R',
   '2016Sci...351...62V',
   '2017ApJ...838..149A',
   '2018MNRAS.475.4011B',
   '2023PASP..135c4101G',
   'ASAS-SN Supernovae'],
  'default': True},
 {'value': '0.021',
  'distance_type': 'redshift',
  'reference': ['2024arXiv240712097A'],
  'default': False},
 {'value': '81.0',
  'distance_type': 'dispersion_measure',
  'unit': 'km/s',
  'reference': ['2017MNRAS.471.1694W', '2024MNRAS.527.2452M'],
  'default': True},
 {'value': '90.7',
  'distance_type': 'comoving',
  'unit': 'Mpc',
  'reference': ['2012PASP..124..668Y',
   '2014ATel.6777....1J',
   '2015arXiv150701598H',
   '2016A&A...594A..13P',
   '2017ApJ...835...64G',
   '2017ApJ...838..149A',
   'ASA

Just like most of the properties here, it is a list of distances with some keywords that tell you about it. 

To work with this, we can simply put it in a pandas dataframe and filter it accordingly. Say you only want luminosity distances, we can then filter that pandas dataframe by the `distance_type` column.

In [14]:
dists = pd.DataFrame(t['distance'])
lum_dists = dists[dists.distance_type == 'luminosity']
lum_dists

,value,distance_type,reference,default,unit
5,92.6,luminosity,"[2012PASP..124..668Y, 2014ATel.6777....1J, 201...",True,Mpc


Which then gives you a luminosity distance and the references you need to cite when you put it in your paper!

A similar process can be used for the rest of the properties so I won't go through them in detail here but just remember, the Transient objects are basically just fancy python dictionaries!